In [60]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,ConvBatchRRelu1D
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])
# without tanh is worse-off

In [45]:
## change ##
batch_size = 16
clip = .25

Prepare data

In [4]:
train_gen = SequenceGen(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGen(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [ ]:
model = Sequential(ConvBatchLeaky1D(1,4,12,stride=10),
                      DenseBlock2(4),
                      DenseBlock2(8),                    
                      DenseBlock2(16),
                      DenseBlock2(32),
                      Dropout(0.5),
                      DenseBlock2(64),
                      Dropout(0.5),
                      DenseBlock2(128),
                      Dropout(0.5),
                      DenseBlock2(256),
                      Dropout(0.5),
                      DenseBlock2(512),
                      Dropout(0.5),
                      DenseBlock2(1024),
                      Dropout(0.5),
                      ConvBatchLeaky1D(2048,1024,28),
                      Dropout(0.5),
                      LambdaLayer(lambda x:x.squeeze(2)),
                      Linear(1024,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [66]:
# All rrelu
class DenseBlock2(nn.Module):
    # N,D,L to N,2*D,L
    def __init__(self, in_channel):
        super().__init__()
        self.in_channel = in_channel
        self.conv1 = ConvBatchRRelu1D(in_channel,in_channel,3,padding=1)
        self.conv2 = ConvBatchRRelu1D(in_channel*2,in_channel*2,3,stride=2)
        
    def forward(self, x):
        return self.conv2(torch.cat([x,self.conv1(x)],1))
    
model = Sequential(ConvBatchRRelu1D(1,4,12,stride=10),
                      DenseBlock2(4),
                      DenseBlock2(8),                    
                      DenseBlock2(16),
                      DenseBlock2(32),
                      DenseBlock2(64),
                      DenseBlock2(128),
                      DenseBlock2(256),
                      DenseBlock2(512),
                      DenseBlock2(1024),                   
                      ConvBatchRRelu1D(2048,1024,28),
                      LambdaLayer(lambda x:x.squeeze(2)),
                      Linear(1024,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [70]:
model = Sequential(ConvBatchLeaky1D(1,4,12,stride=10),
                      ResidualBlockGLU(4),
                      ResidualBlockGLU(8),                    
                      ResidualBlockGLU(16),
                      ResidualBlockGLU(32),
                      Dropout(0.5),
                      ResidualBlockGLU(64),
                      Dropout(0.5),
                      ResidualBlockGLU(128),
                      Dropout(0.5),
                      ResidualBlockGLU(256),
                      Dropout(0.5),
                      ResidualBlockGLU(512),
                      Dropout(0.5),
                      ResidualBlockGLU(1024),
                      Dropout(0.5),
                      ConvBatchLeaky1D(2048,1024,28),
                      Dropout(0.5),
                      LambdaLayer(lambda x:x.squeeze(2)),
                      Linear(1024,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [71]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [72]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [73]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.5291121074786553, val_loss:2.8727221488952637
epoch:1, train_loss:2.2022549962768188, val_loss:2.7832329273223877
epoch:2, train_loss:2.116436899568026, val_loss:2.929361581802368
epoch:3, train_loss:2.0613399067750344, val_loss:2.8159496784210205
epoch:4, train_loss:2.0224673811059732, val_loss:2.9016876220703125
Training completed in 141.07887482643127s
